In [1]:
from ipywidgets import Layout

from main import init
import pandas as pd

SUBJECT = "covid19"  # the subject of data to fetch
NUMBER = 100  # the number of data to fetch from each source
BUILD_CORPUS = True  # whether to build the corpus or load it from a file

search_engine = init(SUBJECT, NUMBER, BUILD_CORPUS)

2025-01-06 23:36:22 fed-main main[59525] INFO Building corpus
2025-01-06 23:36:22 fed-main prawcore[59525] DEBUG Fetching: GET https://oauth.reddit.com/r/covid19/hot at 1736202982.3838818
2025-01-06 23:36:22 fed-main prawcore[59525] DEBUG Data: None
2025-01-06 23:36:22 fed-main prawcore[59525] DEBUG Params: {'limit': 100, 'raw_json': 1}
2025-01-06 23:36:22 fed-main urllib3.connectionpool[59525] DEBUG Starting new HTTPS connection (1): www.reddit.com:443
2025-01-06 23:36:22 fed-main urllib3.connectionpool[59525] DEBUG https://www.reddit.com:443 "POST /api/v1/access_token HTTP/11" 200 652
2025-01-06 23:36:22 fed-main urllib3.connectionpool[59525] DEBUG Starting new HTTPS connection (1): oauth.reddit.com:443
2025-01-06 23:36:23 fed-main urllib3.connectionpool[59525] DEBUG https://oauth.reddit.com:443 "GET /r/covid19/hot?limit=100&raw_json=1 HTTP/11" 200 43940
2025-01-06 23:36:23 fed-main prawcore[59525] DEBUG Response: 200 (43940 bytes) (rst-217:rem-995.0:used-5 ratelimit) at 1736202983.7

            word  frequency  document frequency
0       sentence      33272               33268
1            the      21619               13005
2             to      19327               12747
3            and      19239               13070
4        clinton      18745               18611
...          ...        ...                 ...
13814   braddock          1                   1
13815         pa          1                   1
13816  fetterman          1                   1
13817     clowns          1                   1
13818     reilly          1                   1

[13819 rows x 3 columns]


In [2]:
from ipywidgets import HTML
import ipywidgets as widgets
from IPython.display import display

label = widgets.Label(value="Search for a term in the corpus")

search_box = widgets.Text(
    value='health',
    placeholder='Type something',
    description='Search:',
    disabled=False,
)

slider = widgets.IntSlider(
    value=10,
    min=0,
    max=20,
    step=1,
    description='Search results:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'}
)

button = widgets.Button(description="Search")

search_strength = widgets.IntSlider(
    value=3,
    min=1,
    max=3,
    step=1,
    description='Search strength:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'}
)

enabled_sources_label = widgets.Label(value="Enabled sources")

source_checkbox_list = []

for source in search_engine.get_distinct_sources_list():
    source_checkbox_list.append(widgets.Checkbox(value=True, description=source, layout=Layout(width='150px')))

k_label = widgets.Label(value="keyword importance (only for strength 3)")

k = widgets.FloatSlider(
    value=1.5,
    min=0,
    max=2,
    step=0.1,
    description='k:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'}
)

b_label = widgets.Label(value="prefer shorter documents (only for strength 3)")

b = widgets.FloatSlider(
    value=0.65,
    min=0,
    max=1,
    step=0.05,
    description='b:',
    disabled=True,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'}
)

simple_output = widgets.Checkbox(
    value=False,
    description='Simple Output',
    disabled=False
)

output = widgets.Output()


def update_sliders(*args):
    if search_strength.value < 3:
        k.disabled = True
        b.disabled = True
    else:
        k.disabled = False
        b.disabled = False


search_strength.observe(update_sliders, 'value')


def on_button_clicked(btn):
    with output:
        output.clear_output()
        enabled_sources_list = [src.description for src in source_checkbox_list if src.value]
        if search_strength.value == 1:
            search_results = search_engine.basic_search(search_box.value, enabled_sources_list)
        elif search_strength.value == 2:
            search_results = search_engine.advanced_search(search_box.value, enabled_sources_list)
        else:
            search_results = search_engine.bm25_search(search_box.value, k.value, b.value, enabled_sources_list)

        if search_results.empty:
            display("No results found")

        with pd.option_context('display.max_colwidth', None):
            if slider.value == 0:
                if simple_output.value:
                    display(search_results[["Body", "Score"]])
                else:
                    display(search_results)
            else:
                if simple_output.value:
                    display(search_results.head(slider.value)[["Body", "Score"]])
                else:
                    display(search_results.head(slider.value))


button.on_click(on_button_clicked)

display(
    widgets.VBox(
        [label,
         search_box,
         enabled_sources_label,
         widgets.VBox(source_checkbox_list),
         slider,
         search_strength,
         k_label,
         k,
         b_label,
         b,
         simple_output,
         button, output]))